In [1]:
import cv2
import numpy as np
from glob import glob
from os.path import join as pjoin
import matplotlib.pyplot as plt
from random import randint
from match_elements.matching import image_similarity

In [2]:
from keras.applications.resnet50 import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

Using TensorFlow backend.


In [7]:
def visualize_img_pair(pair):
    plt.subplot(1,2, 1)
    plt.imshow(pair[0])
    plt.subplot(1,2, 2)
    plt.imshow(pair[1])

In [3]:
pair_dir = '/home/ml/Data/result/visual_testing/matched_compos/cleaned'
img_paths = sorted(glob(pjoin(pair_dir, '*')))

name_pre = ''
pos_img_pairs = []
pos_path_pairs = []
for i, path in enumerate(img_paths):
    if i % 2 == 1:
        if img_paths[i - 1].split('/')[-1].split('_')[0] == path.split('/')[-1].split('_')[0]:
            pos_img_pairs.append([cv2.imread(img_paths[i - 1]), cv2.imread(path)])
            pos_path_pairs.append([img_paths[i - 1], path])

In [4]:
neg_img_pairs = []
neg_path_pairs = []
num = len(pos_img_pairs) - 1
for i, path_pair in enumerate(pos_path_pairs):
    r_id = randint(0, num)
    if r_id == i:
        r_id = (r_id + 1) % num
        
    neg_img_pairs.append([pos_img_pairs[i][0], pos_img_pairs[r_id][1]])
    neg_path_pairs.append([path_pair[0], pos_path_pairs[r_id][1]])

In [5]:
pos_path_pairs

[['/home/ml/Data/result/visual_testing/matched_compos/cleaned/0_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/0_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1008_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1008_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1009_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1009_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1018_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1018_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1019_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1019_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1033_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1033_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1034_a.jpg',
  '/home/ml/Data/result/v

In [6]:
neg_path_pairs

[['/home/ml/Data/result/visual_testing/matched_compos/cleaned/0_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1121_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1008_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/153_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1009_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/1097_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1018_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/674_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1019_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/485_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1033_a.jpg',
  '/home/ml/Data/result/visual_testing/matched_compos/cleaned/78_i.jpg'],
 ['/home/ml/Data/result/visual_testing/matched_compos/cleaned/1034_a.jpg',
  '/home/ml/Data/result/vis

In [19]:
def calc_pairs_similarities(pairs):
    sims = {'dhash':[], 'ssim':[], 'surf':[], 'sift':[], 'resnet':[]}
    for pair in pairs:
        img1, img2 = pair
        sims['dhash'].append(image_similarity(img1, img2, method='dhash'))
        sims['ssim'].append(image_similarity(img1, img2, method='ssim', is_gray=True))
        sims['surf'].append(image_similarity(img1, img2, method='surf', draw_match=False, match_distance_ratio=1.3))
        sims['sift'].append(image_similarity(img1, img2, method='sift', draw_match=False, match_distance_ratio=1.3))
        sims['resnet'].append(image_similarity(img1, img2, method='resnet', resnet_model=resnet_model))
    return sims

In [20]:
pos_sims = calc_pairs_similarities(pos_img_pairs)

In [25]:
neg_sims = calc_pairs_similarities(neg_img_pairs)

In [31]:
def correct_ratio(sim_pos, sim_neg, threshold, method):
    pos_correct = 0
    for sim in sim_pos[method]:
        if sim >= threshold:
            pos_correct += 1
    correct_ratio_pos = pos_correct / len(similarities)
    
    neg_correct = 0
    for sim in sim_neg[method]:
        if sim < threshold:
            neg_correct += 1
    correct_ratio_neg = neg_correct / len(similarities)
    
    print("[%s] Positive Correctness: %.3f; Negative Correctness: %.3f" % (method, correct_ratio_pos, correct_ratio_neg))
    return correct_ratio_pos, correct_ratio_neg

In [32]:
correct_ratio(pos_sims, neg_sims, 0.7, 'dhash')

TypeError: '>=' not supported between instances of 'str' and 'float'

In [26]:
def correct_ratio_positive(similarities, pos_thresh):
    pos_correct = 0
    for sim in similarities:
        if sim > pos_thresh:
            pos_correct += 1
    return pos_correct / len(similarities)

In [27]:
def correct_ratio_negative(similarities, neg_thresh):
    neg_correct = 0
    for sim in similarities:
        if sim < neg_thresh:
            neg_correct += 1
    return neg_correct / len(similarities)

In [29]:
print('Positive correctness ratio for [dhash]', correct_ratio_positive(pos_sims['dhash'], 0.7))
print('Positive correctness for [ssim]', correct_ratio_positive(pos_sims['ssim'], 0.7))
print('Positive correctness for [surf]', correct_ratio_positive(pos_sims['surf'], 0.7))
print('Positive correctness for [sift]', correct_ratio_positive(pos_sims['sift'], 0.7))
print('Positive correctness for [resnet]', correct_ratio_positive(pos_sims['resnet'], 0.85))

Correct ratio for [dhash] 0.45686900958466453
Correct ratio for [ssim] 0.3929712460063898
Correct ratio for [surf] 0.2747603833865815
Correct ratio for [sift] 0.33226837060702874
Correct ratio for [resnet] 0.7507987220447284


In [30]:
print('Negative correctness ratio for [dhash]', correct_ratio_negative(neg_sims['dhash'], 0.7))
print('Negative correctness for [ssim]', correct_ratio_negative(neg_sims['ssim'], 0.7))
print('Negative correctness for [surf]', correct_ratio_negative(neg_sims['surf'], 0.7))
print('Negative correctness for [sift]', correct_ratio_negative(neg_sims['sift'], 0.7))
print('Negative correctness for [resnet]', correct_ratio_negative(neg_sims['resnet'], 0.85))

Negative correctness ratio for [dhash] 0.9840255591054313
Negative correctness for [ssim] 0.9297124600638977
Negative correctness for [surf] 0.8274760383386581
Negative correctness for [sift] 0.9105431309904153
Negative correctness for [resnet] 0.9904153354632588
